In [1]:
from typing import Dict
import os, glob

import numpy as np
import pyvista as pv
import pymeshfix

from sire.reconstruct.mesh_merger import MeshSDFMerger

In [ ]:
def euler_characteristic(poly: pv.PolyData):
    return poly.n_points - poly.extract_all_edges().n_lines + poly.n_faces_strict

### Scrapping contours utility

Sometimes we want to scrap last/first few contours tracked, it can be done with this utility.

In [ ]:
def scrap_contours(poly, n_contours, num_points=128, front=True):
    if front:
        points = poly.points.reshape(-1, 128, 3)[n_contours:]
    else:
        points = poly.points.reshape(-1, 128, 3)[:-n_contours]

    contour_lines = np.array([[i, (i + 1) % num_points] for i in range(num_points)])
    all_contour_lines = np.concatenate([contour_lines + i * num_points for i in range(len(points))])
    flat_lines = np.c_[2 * np.ones(len(all_contour_lines))[:, None], all_contour_lines].flatten().astype(int)

    return pv.PolyData(points.reshape(-1, 3), lines=flat_lines)

In [ ]:
root_dir = ""
sample = ""
filename = "contour_iliac_left.vtp"

poly = pv.PolyData(os.path.join(root_dir, sample, "contour/lumen", filename))
scrap_contours(poly, 12, front=False).plot()

### Mesh merging utility
Once all the branches are tracked and meshed we might want blend them together into one mesh, it can be done with this utility.

In [ ]:
def merge_meshes(root_dir: str, configs: Dict[str, str]):
    for name, names in configs.items():
        paths = [path for filename in names for path in glob.glob(os.path.join(root_dir, sample, f"*{filename}*"))]
        mesh_list = [pv.read(path) for path in set(paths)]

        merger = MeshSDFMerger()
        mesh = merger.run(mesh_list, voxel_size=0.15, margin=5, k=0.02, verbose=False)
        mesh = mesh.fill_holes(40)

        euler = euler_characteristic(mesh)

        if euler != 2:
            mfix = pymeshfix.MeshFix(mesh)
            mfix.repair()
            mesh = mfix.mesh

        euler = euler_characteristic(mesh)

        if euler == 2:
            os.makedirs(os.path.join("merged", sample), exist_ok=True)
            mesh.save(f"merges/{sample}/{name}.vtp")
        else:
            print(f"{sample}/{name}:", euler)

In [ ]:
root_dir = ""
configs = {"AAA-full": ["full", "renal", "iliac"]}

merge_meshes(root_dir, configs)

### Centerline clipping utility

If the branches were pruned for reconstruction we might want to clip the centerlines as well, it can be done with this utility.

In [ ]:
def clip_centerline(mesh_file: str, centerline_dir: str, output_dir: str):
    samplename = mesh_file.split("/")[-2]
    mesh = pv.read(mesh_file)

    os.makedirs(os.path.join(output_dir, samplename), exist_ok=True)

    for centerline_file in glob.glob(f"{centerline_dir}/*.vtp"):
        filename = os.path.basename(centerline_file)
        centerline = pv.read(centerline_file)

        enclosed_points = centerline.select_enclosed_points(mesh)["SelectedPoints"]
        clipped_points = centerline.points[enclosed_points.astype(bool)]
        clipped_lines = np.array([(2, i, i + 1) for i in range(len(clipped_points) - 1)]).flatten()
        clipped_centerline = pv.PolyData(clipped_points, lines=clipped_lines)

        clipped_centerline.save(os.path.join(output_dir, samplename, filename))

In [ ]:
filename = ""
centerline_dir = "centerlines"
mesh_dir = "meshes"
output_dir = "centerlines-clipped"

clip_centerline(
    os.path.join(root_dir, mesh_dir, filename, "full.vtp"),
    os.path.join(root_dir, centerline_dir, filename),
    os.path.join(root_dir, output_dir)
)